In [1]:
#init
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig

#tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")

In [2]:
#test tokenizer
#tokenizer("The first computer program that is programmed always says: Hello")

In [3]:
# Pyro-X2/casual_lm_model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig
from transformers import AutoModelForCausalLM
# checkpoint = "openai-community/gpt2-medium"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForCausalLM.from_pretrained(checkpoint)

In [ ]:
tokenizer("The first computer program that is programmed always says: Hello")

In [5]:
prompt = "The first computer program that is programmed always says: Hello"
inputs = tokenizer(prompt, return_tensors="pt")

In [ ]:
outputs = model.generate(**inputs, max_new_tokens=3)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [4]:
checkpoint = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint) 

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [5]:
chat = [{"role": "system", "content": "You are a sassy, wise-cracking robot as imagined by Hollywood circa 1986."},
{"role": "user", "content": "Hey, can you tell me any fun things to do in New York?"},]
tokenizer.chat_template
#tokenizer.chat_template = "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}"
# tokenizer.chat_template

"{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"

In [6]:
tokenized_chat = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt")
print(tokenizer.decode(tokenized_chat[0]))

<|system|>
You are a sassy, wise-cracking robot as imagined by Hollywood circa 1986.</s> 
<|user|>
Hey, can you tell me any fun things to do in New York?</s> 
<|assistant|>



In [7]:
output = model.generate(tokenized_chat, pad_token_id=tokenizer.eos_token_id, do_sample=True, top_k=5, max_new_tokens=1024)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [8]:
response = tokenizer.decode(output[0])
print(response)
tokenizer
model.config

<|system|>
You are a sassy, wise-cracking robot as imagined by Hollywood circa 1986.</s> 
<|user|>
Hey, can you tell me any fun things to do in New York?</s> 
<|assistant|>
Sure thing! New York City, also known as the Big Apple, has plenty of fun things to do. Some popular activities include:

1. Explore Central Park: Central Park is a vast green space in the heart of Manhattan. You can stroll, bike, or take a horse-drawn carriage ride through its winding paths, enjoy a picnic, or visit the Central Park Zoo.

2. Visit the Empire State Building: No trip to New York would be complete without seeing the iconic Empire State Building. Go to the top and enjoy the breathtaking view of the city skyline.

3. Check out Times Square: Times Square is a bustling area in Midtown Manhattan, known for its bright lights, billboards, and crowded streets. You'll find plenty of shops, theaters, and restaurants here.

4. Walk across the Brooklyn Bridge: The Brooklyn Bridge connects Manhattan and Brooklyn. 

MistralConfig {
  "_name_or_path": "HuggingFaceH4/zephyr-7b-beta",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 2,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.44.0",
  "use_cache": true,
  "vocab_size": 32000
}

In [ ]:
# print(tokenizer.batch_decode(output, skip_special_tokens=True))
#response = response.strip('\n')
def process_response(chat_response):
    response_first_split = chat_response.split('</s>')
    #print(response_first_split)
    role=""
    content=""
    for rsp in response_first_split:
        for rsps in rsp.split('>'):
            if '|' in rsps:
                role = rsps.split('|')[-2]
    #             print(role)
            else:
                content = rsps.strip()
    #             print(content)
    #chat.append({"role":role,"content":content})
#     print("role: "+role+" content: "+content)
    return role, content

In [ ]:
# Let's chat for 5 lines
chat_history = chat[:]
role, content = process_response(response)
chat_history.append({"role":role,"content":content})
#print(chat_history)
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
#     print(input(">> User:"))
#     user_ask = [{"role":"user", "content": input("content: ")}]
#     print(user_ask)
    chat_history.append({"role":"user", "content": input("content: ")})
#     print(chat_history)
    tokenized_chat_history = tokenizer.apply_chat_template(chat_history, tokenize=True, add_generation_prompt=True, return_tensors="pt")
#     print(tokenizer.decode(tokenized_chat[0]))
    output_history = model.generate(tokenized_chat_history, max_new_tokens=256, pad_token_id=tokenizer.eos_token_id)
    response_history = tokenizer.decode(output_history[0])
    print(response_history)
    role, content = process_response(response_history)
    chat_history.append({"role":role,"content":content})
#     new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)
